# Import libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm 
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import joblib
from tqdm.notebook import tqdm
from catboost import CatBoostClassifier

# Data Import

In [ ]:
edges = pd.read_csv('/Private/DataScience/Python/Fintech Hackaton Tochka/edges.csv')
ids = pd.read_csv('/Private/DataScience/Python/Fintech Hackaton Tochka/ids.csv')
vertices = pd.read_csv('/Private/DataScience/Python/Fintech Hackaton Tochka/vertices wth 5 neigh.csv') # preprocessed vertices with 5 KNN

In our graph we have 1534749 vertices (nodes)

In [ ]:
len(vertices) 

On this step transforming our features to categorical using LabelEncoder and standardizing them using StandardScaler

In [ ]:
vertices['main_okved'] = vertices['main_okved'].astype(str)
vertices['region_code'] = vertices['region_code'].astype(str)

In [ ]:
le = LabelEncoder()
vertices['company_type'] = le.fit_transform(vertices['company_type'])
vertices['main_okved'] = le.fit_transform(vertices['main_okved'])
vertices['region_code'] = le.fit_transform(vertices['region_code'])

In [ ]:
ss = StandardScaler()
vertices['company_type'] = ss.fit_transform(vertices['company_type'].values.reshape(-1, 1))
vertices['main_okved'] = ss.fit_transform(vertices['main_okved'].values.reshape(-1, 1))
vertices['region_code'] = ss.fit_transform(vertices['region_code'].values.reshape(-1, 1))

In [ ]:
vertices.head()

In [ ]:
vertices.set_index('id', inplace=True)

Creating table for results.

In [ ]:
result = pd.DataFrame(columns=['id_1', 'id_2','preds'])

In [ ]:
edges

In [ ]:
edges.drop(["value","n_transactions"], axis=1, inplace=True)

In [ ]:
from numpy import genfromtxt

In [ ]:
no_edges = genfromtxt('no edges.csv', delimiter=',')
no_edges = no_edges[1:].astype(int)
no_edges

In [ ]:
no_edges[0]

In [ ]:
vertices.loc[no_edges]

In [ ]:
import random
from catboost import Pool

In [ ]:
# учтем то, что вершина i может быть как среди id_1, так и среди id_2
df1 = edges[edges['id_1'] == i].reset_index()
df2 = edges[edges['id_2'] == i].reset_index()

df = df1[['id_2', 'id_1']].rename(columns={'id_1':'id_2', 'id_2':'id_1'}).append(df2[['id_1', 'id_2']])
df['target'] = 1

df = vertices.join(df.set_index('id_1')['target']).fillna(0)

#training set
'''indexes = df.loc[df.target != 1].index
train_data = pd.DataFrame(columns=df.columns)
#train_data
for j in tqdm(range(round(len(df.loc[df.target == 1])*0.7))):
    train_data = train_data.append(df.loc[random.choice(indexes)], sort=False)
train_data = train_data.append(df.loc[df.target == 1].iloc[:round(len(df.loc[df.target == 1])*0.7)], sort=False)

#evaluation set
eval_indx = df.loc[df.target != 1].drop(train_data.index,errors='ignore').index
eval_data = pd.DataFrame(columns=df.columns) #check number

for q in tqdm(range(round(len(df.loc[df.target == 1])*0.3))):
    eval_data = eval_data.append(df.loc[random.choice(eval_indx)], sort=False)
eval_data = eval_data.append(df.loc[df.target == 1].iloc[round(len(df.loc[df.target == 1])*0.7):], sort=False)

eval_dataset = Pool(eval_data.loc[:,'main_okved':'centrality'], eval_data['target'])'''

X = df.loc[:,'main_okved':'5_nn_5']
y = df['target']


In [ ]:
# соберем датасет из всех возможных вершин
# вершины имеющие в исходных данных ребро с i обозначим 1, остальные 0

#model = CatBoostClassifier(task_type='GPU', iterations=100, eval_metric='AUC')
# все признаки категориальные
  
#model = LGBMClassifier(eval_metric='AUC')
model = XGBClassifier(objective ='binary:logistic', n_jobs=-1)

In [ ]:
model.fit(X, y)

In [ ]:
preds = model.predict_proba(X.loc[no_edges])[:,1]

In [ ]:
preds.astype(float)

In [ ]:
df = df.loc[no_edges]

In [ ]:
df['preds'] = preds
df['id_2'] = ids.id.iloc[0]
df.drop(i, inplace=True, errors="ignore")
df.drop(edges[edges['id_2'] == i].id_1.values, inplace=True, errors="ignore")
df.drop(edges[edges['id_1'] == i].id_2.values, inplace=True, errors="ignore")

In [ ]:
df.head()

In [ ]:
#vertices = vertices.merge(degree, how='inner', on='id')

In [ ]:
vertices_orig = pd.read_csv("/Private/DataScience/Python/Fintech Hackaton Tochka/vert no norm 5 nbr.csv")

In [ ]:
vertices_orig.head()

In [ ]:
vertices_orig.set_index('id')

In [ ]:
vertices['company_type'] = vertices_orig['company_type'].astype(str)
vertices['main_okved'] = vertices_orig['main_okved'].astype(str)
vertices['region_code'] = vertices_orig['region_code'].astype(str)

In [ ]:
vertices.head()

In [ ]:
result = pd.DataFrame(columns=['id_1', 'id_2','preds'])

In [ ]:
for i in tqdm(ids.id):
    # соберем датасет из всех возможных вершин
    # вершины имеющие в исходных данных ребро с i обозначим 1, остальные 0
    # учтем то, что вершина i может быть как среди id_1, так и среди id_2
    df1 = edges[edges['id_1'] == i].reset_index()
    df2 = edges[edges['id_2'] == i].reset_index()

    df = df1[['id_2', 'id_1']].rename(columns={'id_1':'id_2', 'id_2':'id_1'}).append(df2[['id_1', 'id_2']])
    df['target'] = 1
    
    df = vertices.join(df.set_index('id_1')['target']).fillna(0)

    #training set
    ''' indexes = df.loc[df.target != 1].index
    train_data = pd.DataFrame(columns=df.columns)
    #train_data
    for j in tqdm(range(round(len(df.loc[df.target == 1])*0.7))):
        train_data = train_data.append(df.loc[random.choice(indexes)], sort=False)
    train_data = train_data.append(df.loc[df.target == 1].iloc[:round(len(df.loc[df.target == 1])*0.7)], sort=False)

    #evaluation set
    eval_indx = df.loc[df.target != 1].drop(train_data.index,errors='ignore').index
    eval_data = pd.DataFrame(columns=df.columns) #check number
    
    for q in tqdm(range(round(len(df.loc[df.target == 1])*0.3))):
        eval_data = eval_data.append(df.loc[random.choice(eval_indx)], sort=False)
    eval_data = eval_data.append(df.loc[df.target == 1].iloc[round(len(df.loc[df.target == 1])*0.7):], sort=False)

    eval_dataset = Pool(eval_data.loc[:,'main_okved':'5_nn_5'], eval_data['target'])
    '''
    X = df.loc[:,'main_okved':'5_nn_5']
    y = df['target']
    

    model = CatBoostClassifier(task_type='GPU', iterations=100, eval_metric='AUC')
    #model_lightgbm = LGBMClassifier(eval_metric='AUC',is_unbalance=True,boosting='gbdt')   
    # все признаки категориальные

    model.fit(X, y,cat_features=[0,1,2,3,4,5])
    print("CatBoost fit ready")
    df = df.loc[no_edges]
  #  df['preds_cat'] = model.predict_proba(X.loc[no_edges])[:,1]
 #   df['preds_lightgbm'] = model_lightgbm.predict_proba(X.loc[no_edges])[:,1]
#    df['preds'] = df['preds_lightgbm']+df['preds_cat']
    df['preds'] = model.predict_proba(X.loc[no_edges])[:,1]

    df['id_2'] = i
    df.drop(i, inplace=True, errors="ignore")
    df.drop(edges[edges['id_2'] == i].id_1.values, inplace=True, errors="ignore")
    df.drop(edges[edges['id_1'] == i].id_2.values, inplace=True, errors="ignore")
    
    
    # возьмем первую 1000 предсказанных ребер, исключив те, про которые мы уже знали
    #res = 
    #res.columns = ['id_1', 'id_2']
    
    result = result.append(df[df['target'] != 1].sort_values(by='preds', ascending=False).iloc[:3000].reset_index()[['id', 'id_2','preds']], ignore_index=True, sort=False)

In [ ]:
result_new.to_csv('/Private/DataScience/Python/Fintech Hackaton Tochka/result_new.csv')

In [ ]:
len(result)

In [ ]:
result = result.loc[result['id'] != result['id_2']]

In [ ]:
result.to_csv('/Private/DataScience/Python/Fintech Hackaton Tochka/res.csv')

In [ ]:
result = pd.read_csv('/Private/DataScience/Python/Fintech Hackaton Tochka/results 20k only sum.csv')

In [ ]:
result.sort_values(by='preds', ascending=False, inplace=True)

In [ ]:
result['id_1'] = result['id']

In [ ]:
result.head()

In [ ]:
result.loc['id_1']

In [ ]:
result.drop(['id','preds'], axis=1,inplace=True)

In [ ]:
result.head()

In [ ]:
import numpy as np

In [ ]:
result_new = pd.DataFrame(np.sort(result[['id_1','id_2']].values, axis=1), columns=result[['id_1','id_2']].columns).drop_duplicates()

In [ ]:
result_new.head()

In [ ]:
len(result_new)

In [ ]:
result_new.id_1 = result_new.id_1.astype(int)
result_new.id_2 = result_new.id_2.astype(int)

In [ ]:
result_new.iloc[:100000].to_csv('/Private/DataScience/Python/Fintech Hackaton Tochka/result_new.csv')

In [ ]:
#!pip install xgboost

In [ ]:
#!pip install lightgbm

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
from xgboost import XGBClassifier

# Graph Analysis

In [ ]:
import networkx as nx

In [ ]:
g = nx.from_pandas_edgelist(edges, source='id_1', target='id_2') 

In [ ]:
degree_dict = dict(g.degree(g.nodes()))
nx.set_node_attributes(g, degree_dict, 'degree')

In [ ]:
errors = []
degrees = []
for j in tqdm(vertices.index):
    try:
        degrees.append([j,g.nodes[j]['degree']])
    except:
    #print('Error',i)
        errors.append(j)
degree = pd.DataFrame(degrees, columns=['id','degree'])
degree.set_index('id', inplace=True)
vertices = vertices.merge(degree, how='inner', on='id')

In [ ]:
vertices['degree'] = ss.fit_transform(vertices['degree'].values.reshape(-1, 1))

In [ ]:
vertices

# Vertices

In [ ]:
import numpy as np

In [ ]:
vertices.head()

In [ ]:
%time nbrs=NearestNeighbors(n_neighbors=4, algorithm='kd_tree', n_jobs=4).fit(np.array(vertices.values))

In [ ]:
distances, indices = nbrs.kneighbors(np.array(vertices.values))

In [ ]:
vertices['1_nearest_neigbour'] = [indices[r][0] for r in range(len(vertices.index))]
vertices['2_nearest_neigbour'] = [indices[r][1] for r in range(len(vertices.index))]
vertices['3_nearest_neigbour'] = [indices[r][2] for r in range(len(vertices.index))]
vertices['4_nearest_neigbour'] = [indices[r][3] for r in range(len(vertices.index))]

In [ ]:
vertices.head()

In [ ]:
vertices.to_csv('/Private/DataScience/Python/Fintech Hackaton Tochka/vertices_normalized with neighbours reworked.csv')

In [ ]:
vertices['1_nearest_neigbour'] = le.fit_transform(vertices['1_nearest_neigbour'])
vertices['2_nearest_neigbour'] = le.fit_transform(vertices['2_nearest_neigbour'])
vertices['3_nearest_neigbour'] = le.fit_transform(vertices['3_nearest_neigbour'])
vertices['4_nearest_neigbour'] = le.fit_transform(vertices['4_nearest_neigbour'])

In [ ]:
df1 = edges[edges['id_1'] == ids.id.iloc[0]].reset_index()
df2 = edges[edges['id_2'] == ids.id.iloc[0]].reset_index()

df = df1[['id_2', 'id_1']].rename(columns={'id_1':'id_2', 'id_2':'id_1'}).append(df2[['id_1', 'id_2']])
df['target'] = 1

df = vertices.join(df.set_index('id_1')['target']).fillna(0)

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
X = df[['main_okved', 'region_code', 'company_type','degree','1_nearest_neigbour','2_nearest_neigbour','3_nearest_neigbour','4_nearest_neigbour']]
y = df['target']
model = CatBoostClassifier(task_type='GPU',iterations=100, verbose=True)


In [ ]:
model.fit(X, y)
preds = model.predict_proba(X)[:,1]

In [ ]:
X = df[['main_okved', 'region_code', 'company_type','degree','1_nearest_neigbour','2_nearest_neigbour','3_nearest_neigbour','4_nearest_neigbour']]
y = df['target']
model = CatBoostClassifier(task_type='GPU',iterations=100, verbose=True)


# X = df[['main_okved', 'region_code', 'company_type']]
#y = df['target']

#model = CatBoostClassifier(iterations=100, verbose=False)
#cat_features = [0,1,2] # все признаки категориальные

#X.drop(edges[edges['id_2'] == i].id_1.values, inplace=True)
#X.drop(edges[edges['id_1'] == i].id_2.values, inplace=True)
#df.drop(edges[edges['id_2'] == i].id_1.values, inplace=True)
#df.drop(edges[edges['id_1'] == i].id_2.values, inplace=True)


df['preds'] = preds
df['id_2'] = ids.id.iloc[0]

In [ ]:
df.head()

In [ ]:
df.sort_values(by='preds', ascending=False).iloc[:1000]

In [ ]:
len(df)

In [ ]:
df.drop(ids.id,errors='ignore')

In [ ]:
result = pd.DataFrame(columns=['id_1', 'id_2'])

In [ ]:
for i in tqdm(ids.id):
    df1 = edges[edges['id_1'] == i].reset_index()
    df2 = edges[edges['id_2'] == i].reset_index()

    df = df1[['id_2', 'id_1']].rename(columns={'id_1':'id_2', 'id_2':'id_1'}).append(df2[['id_1', 'id_2']])
    df['target'] = 1

    df = vertices.join(df.set_index('id_1')['target']).fillna(0)
    df['id_1']=df.index
    df.drop(ids.id.values, errors='ignore',inplace=True)
   
    X = df[['main_okved', 'region_code', 'company_type','degree','1_nearest_neigbour','2_nearest_neigbour','3_nearest_neigbour','4_nearest_neigbour']]
    y = df['target']
    model = CatBoostClassifier(task_type='GPU',iterations=100, verbose=False)
    model.fit(X, y)
    preds = model.predict_proba(X)[:,1]
    df['preds'] = preds
    df['id_2'] = i

    # возьмем первую 1000 предсказанных ребер, исключив те, про которые мы уже знали
    res = df[(df['target'] != 1) & (df.index!=i)].sort_values(by='preds', ascending=False).iloc[:1000]
    res.columns = ['df1','main_okved', 'region_code', 'company_type','degree','1_nearest_neigbour','2_nearest_neigbour','3_nearest_neigbour','4_nearest_neigbour', 'target','prob','id_2']
    

    result = result.append(res, ignore_index=True, sort=False)

# Other

Model check

In [ ]:
#edges['id_1'] = edges['id_1'].astype(str)
#edges['id_2'] = edges['id_2'].astype(str)

In [ ]:
edges.describe()

In [ ]:
ids.id.iloc[0]

In [ ]:
statistics = []

In [ ]:
for i in tqdm.tqdm(ids.id):
    statistics.append([i,len(edges[edges['id_1'] == i])])

In [ ]:
statistics[0:10]

In [ ]:
plt.figure()
plt.subplot(2,1,1)
plt.hist(edges.n_transactions)
plt.title("Распределение количества транзакций")
plt.subplot(2,1,2)
plt.hist(edges.value)
plt.title("Распределение сумм транзакций")
plt.show()

In [ ]:
X = edges.iloc[:,[0,2,3]]
y = edges.iloc[:,1]

# Model whole

One iteration

Adding new features
* graph (degree)
* оквед_1, оквед_2
* метомодель
* KNN (номер кластера)
* близжайшие соседи
* кратчайшие пути

#  CatBoost Model

One Iteration

In [ ]:
df1 = edges[edges['id_1'] == ids.id.iloc[0]].reset_index()
df2 = edges[edges['id_2'] == ids.id.iloc[0]].reset_index()

df = df1[['id_2', 'id_1']].rename(columns={'id_1':'id_2', 'id_2':'id_1'}).append(df2[['id_1', 'id_2']])
df['target'] = 1

df = vertices.set_index('id').join(df.set_index('id_1')['target']).fillna(0)
#from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['company_type'] = le.fit_transform(df['company_type'])
df['main_okved'] = le.fit_transform(df['main_okved'])

#Degree
errors = []
degrees = []
for j in df.index:
    try:
        degrees.append([j,g.nodes[j]['degree']])
    except:
    #print('Error',i)
        errors.append(j)

degree = pd.DataFrame(degrees, columns=['id','degree'])
degree.set_index('id', inplace=True)
df = df.merge(degree, how='inner', on='id')

#KNN
#nbrs = NearestNeighbors(n_neighbors=4, algorithm='kd_tree', n_jobs=4).fit(df[['main_okved','region_code','company_type','degree']].to_numpy())
#distances, indices = nbrs.kneighbors(df[['main_okved','region_code','company_type','degree']].to_numpy())
#df['1_nearest_neigbour'] = [indices[r][0] for r in range(len(df.index))]
#df['2_nearest_neigbour'] = [indices[r][1] for r in range(len(df.index))]
#df['3_nearest_neigbour'] = [indices[r][2] for r in range(len(df.index))]
#df['4_nearest_neigbour'] = [indices[r][3] for r in range(len(df.index))]

X = df[['main_okved', 'region_code', 'company_type','degree','1_nearest_neigbour','4_nearest_neigbour','2_nearest_neigbour','3_nearest_neigbour']]
y = df['target']
model = CatBoostClassifier(task_type='GPU',iterations=100, verbose=False)
cat_features = [0,1,2,4,5,6,7] 

# X = df[['main_okved', 'region_code', 'company_type']]
#y = df['target']

#model = CatBoostClassifier(iterations=100, verbose=False)
#cat_features = [0,1,2] # все признаки категориальные

model.fit(X, y, cat_features)

preds = model.predict_proba(X)[:,1]

df['preds'] = preds
df['id_2'] = ids.id.iloc[0]

# возьмем первую 1000 предсказанных ребер, исключив те, про которые мы уже знали
res = df[df['target'] != 1].sort_values(by='preds', ascending=False).iloc[:1000].reset_index()[['id', 'id_2']]
res.columns = ['id_1', 'id_2']

result = result.append(res, ignore_index=True, sort=False)

In [ ]:
df1 = edges[edges['id_1'] == ids.id.iloc[0]].reset_index()
df2 = edges[edges['id_2'] == ids.id.iloc[0]].reset_index()

df = df1[['id_2', 'id_1']].rename(columns={'id_1':'id_2', 'id_2':'id_1'}).append(df2[['id_1', 'id_2']])
df['target'] = 1

df = vertices.set_index('id').join(df.set_index('id_1')['target']).fillna(0)
#from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['company_type'] = le.fit_transform(df['company_type'])
df['main_okved'] = le.fit_transform(df['main_okved'])
df['main_okved'] = le.fit_transform(df['main_okved'])

#Degree
errors = []
degrees = []
for j in df.index:
    try:
        degrees.append([j,g.nodes[j]['degree']])
    except:
    #print('Error',i)
        errors.append(j)

degree = pd.DataFrame(degrees, columns=['id','degree'])
degree.set_index('id', inplace=True)
df = df.merge(degree, how='inner', on='id')

#KNN
#nbrs = NearestNeighbors(n_neighbors=4, algorithm='kd_tree', n_jobs=4).fit(df[['main_okved','region_code','company_type','degree']].to_numpy())
#distances, indices = nbrs.kneighbors(df[['main_okved','region_code','company_type','degree']].to_numpy())
#df['1_nearest_neigbour'] = [indices[r][0] for r in range(len(df.index))]
#df['2_nearest_neigbour'] = [indices[r][1] for r in range(len(df.index))]
#df['3_nearest_neigbour'] = [indices[r][2] for r in range(len(df.index))]
#df['4_nearest_neigbour'] = [indices[r][3] for r in range(len(df.index))]

X = df[['main_okved', 'region_code', 'company_type','degree']]
y = df['target']
model = CatBoostClassifier(task_type='GPU',iterations=100, verbose=False)
cat_features = [0,1,2] 

# X = df[['main_okved', 'region_code', 'company_type']]
#y = df['target']

#model = CatBoostClassifier(iterations=100, verbose=False)
#cat_features = [0,1,2] # все признаки категориальные

model.fit(X, y, cat_features)

preds = model.predict_proba(X)[:,1]

df['preds'] = preds
df['id_2'] = ids.id.iloc[0]

# возьмем первую 1000 предсказанных ребер, исключив те, про которые мы уже знали
res = df[df['target'] != 1].sort_values(by='preds', ascending=False).iloc[:1000].reset_index()[['id', 'id_2']]
res.columns = ['id_1', 'id_2']

result = result.append(res, ignore_index=True, sort=False)


In [ ]:
from tqdm import tqdm_notebook

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
result = pd.DataFrame(columns=['id_1', 'id_2'])

Without neighbours

In [ ]:
for i in tqdm(ids.id):
    df1 = edges[edges['id_1'] == i].reset_index()
    df2 = edges[edges['id_2'] == i].reset_index()

    df = df1[['id_2', 'id_1']].rename(columns={'id_1':'id_2', 'id_2':'id_1'}).append(df2[['id_1', 'id_2']])
    df['target'] = 1

    df = vertices.set_index('id').join(df.set_index('id_1')['target']).fillna(0)
    #from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    #df['company_type'] = le.fit_transform(df['company_type'])
    #df['main_okved'] = le.fit_transform(df['main_okved'])
    
    #Degree
    errors = []
    degrees = []
    for j in tqdm(df.index):
        try:
            degrees.append([j,g.nodes[j]['degree']])
        except:
        #print('Error',i)
            errors.append(j)

    degree = pd.DataFrame(degrees, columns=['id','degree'])
    degree.set_index('id', inplace=True)
    df = df.merge(degree, how='inner', on='id')
    df.drop(ids.id.values, inplace=True)

    #KNN
    #nbrs = NearestNeighbors(n_neighbors=4, algorithm='kd_tree', n_jobs=4).fit(df[['main_okved','region_code','company_type','degree']].to_numpy())
    #distances, indices = nbrs.kneighbors(df[['main_okved','region_code','company_type','degree']].to_numpy())
    #df['1_nearest_neigbour'] = [indices[r][0] for r in range(len(df.index))]
    #df['2_nearest_neigbour'] = [indices[r][1] for r in range(len(df.index))]
    #df['3_nearest_neigbour'] = [indices[r][2] for r in range(len(df.index))]
    #df['4_nearest_neigbour'] = [indices[r][3] for r in range(len(df.index))]

    X = df[['main_okved', 'region_code', 'company_type','degree']]
    y = df['target']
    model = CatBoostClassifier(task_type='GPU',iterations=100, verbose=False)
    cat_features = [0,1,2] 

    # X = df[['main_okved', 'region_code', 'company_type']]
    #y = df['target']

    #model = CatBoostClassifier(iterations=100, verbose=False)
    #cat_features = [0,1,2] # все признаки категориальные

    model.fit(X, y, cat_features)
    #X.drop(edges[edges['id_2'] == i].id_1.values, inplace=True)
    #X.drop(edges[edges['id_1'] == i].id_2.values, inplace=True)

    preds = model.predict_proba(X)[:,1]
    #df.drop(edges[edges['id_2'] == i].id_1.values, inplace=True)
    #df.drop(edges[edges['id_1'] == i].id_2.values, inplace=True)


    df['preds'] = preds
    df['id_2'] = i
    #df['prob'] = model.predict_proba(X)[:,0]

    # возьмем первую 1000 предсказанных ребер, исключив те, про которые мы уже знали
    #res['prob'] = model.predict_proba(X)[:,0]
    res = df[(df['target'] != 1) & (df.index!=i)].sort_values(by='preds', ascending=False).iloc[:1000].reset_index()[['id', 'id_2']]
    res.columns = ['id_1', 'id_2']
    

    result = result.append(res, ignore_index=True, sort=False)

In [ ]:
model.predict_proba(X)[:,0]

In [ ]:
df.head()

In [ ]:
df.sort_values(by="preds")

In [ ]:
#result.to_csv('/Private/result1.csv', index=False)

In [ ]:
len(result)

One Iteration Disambl

In [ ]:
df1 = edges[edges['id_1'] == ids.id.iloc[0]].reset_index()
df2 = edges[edges['id_2'] == ids.id.iloc[0]].reset_index()

df = df1[['id_2', 'id_1']].rename(columns={'id_1':'id_2', 'id_2':'id_1'}).append(df2[['id_1', 'id_2']])
df['target'] = 1

df = vertices.set_index('id').join(df.set_index('id_1')['target']).fillna(0)
#from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#df['company_type'] = le.fit_transform(df['company_type'])
#df['main_okved'] = le.fit_transform(df['main_okved'])
#df['main_okved'] = le.fit_transform(df['main_okved'])

In [ ]:
errors = []
degrees = []
for i in tqdm(df.index):
    try:
        degrees.append([i,g.nodes[i]['degree']])
    except:
        #print('Error',i)
        errors.append(i)
    

In [ ]:
degree = pd.DataFrame(degrees, columns=['id','degree'])
degree.set_index('id', inplace=True)

In [ ]:
degree.head()

In [ ]:
#df.drop(['degree'],axis=1, inplace=True)

In [ ]:
degree = pd.DataFrame(degrees, columns=['id','degree'])
degree.set_index('id', inplace=True)
df = df.merge(degree, how='inner', on='id')

In [ ]:
df['main_okved'] = df['main_okved'].astype(str)
df['company_type'] = df['company_type'].astype(str)

Nearest Neighbours

In [ ]:
%time nbrs = NearestNeighbors(n_neighbors=2, algorithm='kd_tree', n_jobs=4).fit(df[['main_okved','region_code','company_type','degree']].to_numpy())

In [ ]:
distances, indices = nbrs.kneighbors(df[['main_okved','region_code','company_type','degree']].to_numpy())
df['1_nearest_neigbour'] = [indices[i][0] for i in range(len(df.index))]
df['2_nearest_neigbour'] = [indices[i][1] for i in range(len(df.index))]
df['3_nearest_neigbour'] = [indices[i][2] for i in range(len(df.index))]
df['4_nearest_neigbour'] = [indices[i][3] for i in range(len(df.index))]

In [ ]:
df.head()

In [ ]:
result = pd.DataFrame(columns=['id_1', 'id_2'])

In [ ]:
X = df[['main_okved', 'region_code', 'company_type','degree','1_nearest_neigbour','4_nearest_neigbour','2_nearest_neigbour','3_nearest_neigbour']]
y = df['target']
model = CatBoostClassifier(task_type='GPU',iterations=100, verbose=False)
cat_features = [0,1,2] 

# X = df[['main_okved', 'region_code', 'company_type']]
#y = df['target']

#model = CatBoostClassifier(iterations=100, verbose=False)
#cat_features = [0,1,2] # все признаки категориальные

model.fit(X, y, cat_features)

preds = model.predict_proba(X)[:,1]

df['preds'] = preds
df['id_2'] = ids.id.iloc[0]

# возьмем первую 1000 предсказанных ребер, исключив те, про которые мы уже знали
res = df[df['target'] != 1].sort_values(by='preds', ascending=False).iloc[:1000].reset_index()[['id', 'id_2']]
res.columns = ['id_1', 'id_2']

result = result.append(res, ignore_index=True, sort=False)

In [ ]:
result.head()

In [ ]:
len(result)

K-Means. No K-Means. Long

For each whole data

In [ ]:
result = pd.DataFrame(columns=['id_1', 'id_2'])

In [ ]:
for i in tqdm(ids.id):
    # соберем датасет из всех возможных вершин
    # вершины имеющие в исходных данных ребро с i обозначим 1, остальные 0
    # учтем то, что вершина i может быть как среди id_1, так и среди id_2
    df1 = edges[edges['id_1'] == i].reset_index()
    df2 = edges[edges['id_2'] == i].reset_index()
    

    df = df1[['id_2', 'id_1']].rename(columns={'id_1':'id_2', 'id_2':'id_1'}).append(df2[['id_1', 'id_2']])
    df['target'] = 1
    
    df = vertices.set_index('id').join(df.set_index('id_1')['target']).fillna(0)
    #from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    df['company_type'] = le.fit_transform(df['company_type'])
    df['main_okved'] = le.fit_transform(df['main_okved'])
    #Degree
    errors = []
    degrees = []
    for j in tqdm(df.index):
        try:
            degrees.append([j,g.nodes[j]['degree']])
        except:
        #print('Error',i)
            errors.append(j)
    
    degree = pd.DataFrame(degrees, columns=['id','degree'])
    degree.set_index('id', inplace=True)
    df = df.merge(degree, how='inner', on='id')
    #KNN
    nbrs = NearestNeighbors(n_neighbors=4, algorithm='kd_tree', n_jobs=4).fit(df.to_numpy())
    distances, indices = nbrs.kneighbors(df.to_numpy())
    df['1_nearest_neigbour'] = [indices[r][0] for r in range(len(df.index))]
    df['2_nearest_neigbour'] = [indices[r][1] for r in range(len(df.index))]
    df['3_nearest_neigbour'] = [indices[r][2] for r in range(len(df.index))]
    df['4_nearest_neigbour'] = [indices[r][3] for r in range(len(df.index))]
    
    X = df[['main_okved', 'region_code', 'company_type','degree','1_nearest_neigbour','4_nearest_neigbour','2_nearest_neigbour','3_nearest_neigbour']]
    y = df['target']
    model = CatBoostClassifier(task_type='GPU',iterations=100, verbose=False)
    cat_features = [0,1,2,4,5,6,7] 
    
   # X = df[['main_okved', 'region_code', 'company_type']]
    #y = df['target']
    
    #model = CatBoostClassifier(iterations=100, verbose=False)
    #cat_features = [0,1,2] # все признаки категориальные
    
    model.fit(X, y, cat_features)

    preds = model.predict_proba(X)[:,1]

    df['preds'] = preds
    df['id_2'] = i
    
    # возьмем первую 1000 предсказанных ребер, исключив те, про которые мы уже знали
    res = df[df['target'] != 1].sort_values(by='preds', ascending=False).iloc[:1000].reset_index()[['id', 'id_2']]
    res.columns = ['id_1', 'id_2']
    
    result = result.append(res, ignore_index=True, sort=False)